In [1]:
#| warning: false
#| code-fold: true
#import library
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#render ploty
pio.renderers.default = "notebook"

#import data
us_states=gpd.read_file("../final_data/tl_2020_us_state.shp")
df = pd.read_csv("../final_data/final_data.csv")
original_df = pd.read_csv("../final_data/final_data.csv")
# Initialize the Dash app
app = dash.Dash(__name__)

#normalize data
columns_to_normalize = ['Personal Income', 'Median Rent', 'Median Sale Price',
                        'Walkability', 'Unemployment', 'Purchasing Power',
                        'Violent Crime Rate', 'Non-Violent Crime Rate']

scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Columns where lower values are better
columns_lower_better = ['Median Rent', 'Unemployment', 'Violent Crime Rate', 'Non-Violent Crime Rate']

# Invert scaling for those columns
df[columns_lower_better] = 1 - df[columns_lower_better]

    # Define color scale mapping each political affiliation to a specific color
color_scale = {
    "Democrat": "blue",
    "Republican": "red",
    "Independent": "yellow",
    "Nonpartisan": "grey"
    }

## Introduction

Have you ever dreamed of living in a big city?

Or maybe you already live in one and you are looking for a new place to move to?

Do you feel like you are missing out on something?

Is there something more to life than what you are currently experiencing?

If you answered yes to any of these questions, then you've come to the right place!

This website is dedicated to helping people like you find the perfect city to live in. Whether you are an aspiring artist, a young professional, a graduate student, or a retiree, we have the perfect city for you. Our website is designed to help you find the perfect city based on essential living factors such as cost of living, quality of life, safety, and more. We have taken the most prominent metropolitan areas in the United States and provided their livability based on your personal preferences at your fingertips; so you can make an informed decision about where to live. All you have to do is adjust the sliders to your preferences and we will do the rest. 

The bigger the circle, the better the city is for you! Additionally, we have provided a complementary table that ranks the cities based on your preferences.

**So what are you waiting for? Find your next adventure now!**

In [2]:
# Define the layout of the app
app.layout = html.Div([
    html.H1("Rank by Importance",style={"color": "black"}),
    html.Div([
        html.Div([
            html.Label("Importance of Rent",style={"color": "black"}),
            dcc.Slider(
                id="Median Rent",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Importance of Income",style={"color": "black"}),
            dcc.Slider(
                id="Personal Income",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Importance of Home Sale Price",style={"color": "black"}),
            dcc.Slider(
                id="Median Sale Price",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Importance of Walkablity",style={"color": "black"}),
            dcc.Slider(
                id="Walkable",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Importance of Job Security and Opportunity",style={"color": "black"}),
            dcc.Slider(
                id="Unemployment",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Importance of Purchasing Power",style={"color": "black"}),
            dcc.Slider(
                id="Purchasing Power",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Importance of Violent Crime Rate",style={"color": "black"}),
            dcc.Slider(
                id="Crime Rate",
                min=0,
                max=1,
                step=0.1,
                value=0.5,
                marks={i/10: str(i/10) for i in range(11)},
                tooltip={"always_visible": False, "placement": "bottom"}
            ),
            html.Label("Variable to Visualize", style={"color": "black"}),
            dcc.Dropdown(
                id="variable",
                options=[{"label": i, "value": i} for i in df.columns if i not in ['Latitude', 'Longitude']],
                value=df.columns[0]
            ),
        ], style={"width": "30%", "float": "left"}),
        html.Div([
            dcc.Graph(id="city-graph"),
            dash_table.DataTable(id='table')
        ], style={"width": "70%", "float": "right"}),
    ]),
])

# Define callback to update the graph based on user input
@app.callback(
    [Output("city-graph", "figure"), Output("table", "data"), Output("table", "columns")],
    [Input("Median Rent", "value"),
     Input("Personal Income", "value"),
     Input("Median Sale Price","value"),
     Input("Walkable","value"),
     Input("Unemployment","value"),
     Input("Purchasing Power","value"),
     Input("Crime Rate","value"),
     Input("variable", "value")])
def update_graph(importance_personal_income, importance_median_rent,importance_median_sale_price,importance_walkable,importance_unemployment,importance_purchasing_power,importance_crime, selected_variable):
    # Weighted sum of variables
    df["Weighted_Sum"] = (df["Median Rent"] * importance_personal_income +
                          df["Personal Income"] * importance_median_rent + df["Median Sale Price"]*importance_median_sale_price +df["Walkability"]*importance_walkable+df["Unemployment"]*importance_unemployment+
                          df["Purchasing Power"]*importance_purchasing_power+df["Violent Crime Rate"]*importance_crime)
    # Apply power transformation for better visualization
    df["Weighted_Sum"] = df["Weighted_Sum"] ** 3
    
    # Define color scale mapping each political affiliation to a specific color
    color_scale = {
        "Democrat": "blue",
        "Republican": "red",
        "Independent": "yellow",
        "Nonpartisan": "grey"
    }
    
    # Plotly scatter plot
    fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', scope='usa', size="Weighted_Sum", color=selected_variable,
                     color_discrete_map=color_scale, custom_data=["Region", "Weighted_Sum"])
    
    fig.update_layout(title="Best Metro Areas by Personal Livability Score",
                      mapbox_style="carto-positron",
                      mapbox_zoom=3,
                      mapbox_center={"lat": 37.0902, "lon": -95.7129})
    
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>Weighted Sum: %{customdata[1]:.2f}')
    
    sorted_df = df.sort_values("Weighted_Sum", ascending=False)
    table_data = sorted_df[["Region", "Weighted_Sum"]].to_dict('records')
    table_columns = [{"name": i, "id": i} for i in sorted_df[["Region", "Weighted_Sum"]].columns]
    
    return fig, table_data, table_columns

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True,port=8052)

`N.B. The dropdown menu is available at the bottom left corner of the map. It colour codes the cities based on the categories you select.`

---------

### A closer look at each metro area

Now that you have a general idea of which cities are best for you, let's take a closer look at each metro area. In the visualization below, we will be showing you exactly how each city ranks in terms of a multitude of categories. You simply select which specific category you would like to see and we will show you how each city ranks in that category. The bigger the circle, the higher the amount of that category in that city. Additionally, we have provided a supplementary bar chart that shows you the exact values of each city in that category. If you would like to see the exact values of each city, simply hover over the circle or the bar chart.

In [3]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
    id="variable",
    options=[{"label": i, "value": i} for i in original_df.columns if i not in ['Latitude', 'Longitude', 'Region', 'Political Affiliation']],
    value='Personal Income'
    ),
    dcc.Graph(id="scatter-plot"),
    dcc.Graph(id="bar-chart"),
])

@app.callback(
    [Output("scatter-plot", "figure"),
     Output("bar-chart", "figure")],
    [Input("variable", "value")]
)
def update_graphs(variable):
    scatter_plot = px.scatter_geo(original_df, lat='Latitude', lon='Longitude', scope='usa', size=variable, color=variable)
    bar_chart = px.bar(original_df.sort_values(variable, ascending=False), x='Region', y=variable, color=variable)
    return scatter_plot, bar_chart

if __name__ == "__main__":
    app.run_server(debug=True)

----------

### Viz 3

------------

In [5]:
#| warning: false
#| code-fold: true
df = pd.read_csv("../final_data/final_data.csv")

df.head()

,Region,Personal Income,Median Rent,Median Sale Price,Walkability,Unemployment,Purchasing Power,Latitude,Longitude,Political Affiliation,Average Yearly Temperature(F),Violent Crime Rate,Non-Violent Crime Rate
0,Atlanta,64107,2113.0,357000.0,49.4,3.0,100.89,33.7490,-84.3880,Democrat,62.6,5727.52,4776.43
1,Austin,75119,1937.5,451250.0,50.0,3.7,99.96,30.2672,-97.7431,Democrat,69.3,3612.23,3189.57
2,Baltimore,71420,1981.5,325500.0,44.7,2.8,95.23,39.2904,-76.6122,Democrat,55.6,6997.68,4928.11
3,Boston,94082,3939.5,600000.0,74.3,3.3,91.17,42.3601,-71.0589,Democrat,51.7,2758.22,2089.02
4,Charlotte,65156,1923.5,365000.0,51.7,3.6,103.52,35.2271,-80.8431,Democrat,60.9,3836.39,3815.18


In [ ]:
#| warning: false
#| code-fold: true
# #def east/west
color_scale = {
    "Democrat": "blue",
    "Republican": "red",
    "Independent": "yellow",
    "Nonpartisan": "grey"
    }

def east_west(region):
    west_coast = ['Seattle', 'San Francisco', 'Los Angeles', 'San Diego', 'Portland']
    east_coast = ['Boston', 'New York', 'Washington, DC', 'Miami', 'Philadelphia', 'Atlanta', 'Baltimore', 'Virginia Beach', 'Charlotte', 'Tampa', 'Orlando', 'Miami']
    if region in west_coast:
        return "west"
    elif region in east_coast:
        return 'east'
    else:
        return 'cent'
    
df["coast"] = df["Region"].apply(east_west)

# Sort the DataFrame by 'Year_Fahrenheit' column
# df_sorted = df.sort_values(by='Year_Fahrenheit')


#make df for each part of the country
#east
df_east = df[df['coast'] == 'east']
df_east = df_east.sort_values(by='Average Yearly Temperature(F)')
#west
df_west = df[df['coast'] == 'west']
df_west = df_west.sort_values(by='Average Yearly Temperature(F)')
#cent
df_cent = df[df['coast'] == 'cent']
df_cent = df_cent.sort_values(by='Average Yearly Temperature(F)')

# For average temperature in west region
fig1 = px.bar(df_west, x='Region', y='Average Yearly Temperature(F)', color='Political Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Western Metro Areas')

# For average temperature in central region
fig2 = px.bar(df_cent, x='Region', y='Average Yearly Temperature(F)', color='Political Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Central Metro Areas')
# For average temperature in east region
fig3 = px.bar(df_east, x='Region', y='Average Yearly Temperature(F)', color='Political Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Eastern Metro Areas')

# Create subplots with 1 row and 3 columns
fig = make_subplots(rows=1, cols=3, subplot_titles=('West Coast Metro Areas', 'Central Metro Areas', 'East Coast Metro Areas'))

# Add the plots to the subplots
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=1, col=3)

# Update layout
fig.update_layout(title_text='Average Temperature by Metro Area', yaxis_title='Temperature (F)', showlegend=False)
# Update bar width
fig.update_traces(width=.75)
# Show the plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#walkability

#sort
df = df.sort_values(by='Walkability', ascending=False)
#plot
fig = px.bar(df, x='Region', y='Walkability', color='Political Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Walkability Score', title='Walkability Score by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#walkability
#sort
df = df.sort_values(by='Purchasing Power', ascending=False)
#plot
fig = px.bar(df, x='Region', y='Purchasing Power', color='Political Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Purchasing Power by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#median rent
#sort
df = df.sort_values(by='Median Rent', ascending=False)
#plot
fig = px.bar(df, x='Region', y='Median Rent', color='Political Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Median Rent by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [14]:
print(df.columns)

Index(['Region', 'Personal Income', 'Median Rent', 'Median Sale Price',
       'Walkability', 'Unemployment', 'Purchasing Power', 'Latitude',
       'Longitude', 'Political Affiliation', 'Average Yearly Temperature(F)',
       'Violent Crime Rate', 'Non-Violent Crime Rate', 'coast'],
      dtype='object')


In [ ]:
#| warning: false
#| code-fold: true
#median.sale.price
#sort
df = df.sort_values(by='Median Sale Price', ascending=False)
#plot
fig = px.bar(df, x='Region', y='Median Sale Price', color='Political Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Median Home Sale Price by Metro Area')

#Hide legend
fig.update_traces(showlegend=False)
fig.show()